In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [ ]:
# file paths to mex on Colab
data_file = "sample_data/mex.csv"

In [ ]:
# read all data
data = pd.read_csv(data_file, header=None)
seed =100 #useful for random generators
print(data.shape)
data.head()

# Accessing different parts of the collected data

In [ ]:
# extract class labels from data
y = data.iloc[:, 873]
print(y.shape)

(6312,)


In [ ]:
#extract person IDs
id = data.iloc[:, 0]
print(id.shape)

(6312,)


In [ ]:
# extract features from wrist accelerometer data
acw_x = data.iloc[:, 1:181]
print(acw_x.shape)

(6312, 180)


In [ ]:
# extract features from thigh accelerometer data
act_x = data.iloc[:, 181:361]
print(act_x.shape)

(6312, 180)


In [ ]:
# extract features from pressure mat data
pm_x = data.iloc[:, 361:873]
print(pm_x.shape)

(6312, 512)


In [ ]:
# extract wrist and thigh accelerometer data
wt_x = data.iloc[:, 1:361]
print(wt_x.shape)

(6312, 360)


In [ ]:
# extract wrist, thigh and pressure mat data
wtp_x = data.iloc[:, 1:873]
print(wtp_x.shape)

(6312, 872)


In [ ]:
# a few examples of models - keras learners can also be explored
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier(n_neighbors=5, weights='distance')))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(kernel='rbf', gamma=0.7, C=1.0)))
models.append(('ANN', MLPClassifier()))

# 2.1 Evaluate the role of sensors on classification

In [ ]:
print('Running algorithms for wrist accelerometer data...')

# complete code here

In [ ]:
print('Running algorithms for thigh accelerometer data...')

# complete code here

In [ ]:
print('Running algorithms for pressure mat data...')

# complete code here

In [ ]:
print('Running algorithms for wrist and thigh accelerometer data and pressure mat data...')
#


# complete code here

# 2.2 Evaluate the role of persons data on classification

In [ ]:
#print all unique person Ids
data[0].unique()

In [ ]:
#helper function to split data by Leave-One-Person-Out
def lopo_split(df, indices=[], test_person=None):
    if test_person == None:
        print("Need test person id")
        return
    if len(indices) == 0 | len(indices)>2 | indices[0]<1 | indices[1]>872 | indices[0]>indices[1]:
        print("Need valid indices")
        return

    train_df = df[df[0]!=test_person]
    test_df = df[df[0]==test_person]
    X_train = train_df.iloc[:, indices[0]:indices[1]].to_numpy()
    X_test = test_df.iloc[:, indices[0]:indices[1]].to_numpy()
    y_train = train_df.iloc[:, 873].to_numpy()
    y_test = test_df.iloc[:, 873].to_numpy()

    return X_train, X_test, y_train, y_test


In [ ]:
#testing helper method
#indicies indicate the sensor or sensors needed
X_train, X_test, y_train, y_test = lopo_split(data, indices=[1, 181], test_person=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
print('Running algorithms for wrist accelerometer data...')

# complete code here

In [ ]:
print('Running algorithms for thigh accelerometer data...')

# complete code here

In [ ]:
print('Running algorithms for pressure mat data...')

# complete code here

In [ ]:
print('Running algorithms for wrist and thigh accelerometer data and pressure mat data...')

# complete code here